In [17]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None

In [18]:
# which team will travel the most
# has the most back to backs
# has the most 2/3s, 3/4s, and 4/5s
# has the most high elevation games
# has the most games with less then 24 hours

In [19]:
# game_time_df = pd.read_csv("../data/game_times_bask_ref_2008-2024.csv")
game_time_df = pd.read_csv("../data/games_bask_ref_2008-2024.csv")

# get day of week
game_time_df["day_of_week"] = game_time_df["Date"].str.split(",").str[0]

# filter for season
# game_time_df = game_time_df[game_time_df["season_end_year"] == 2024].reset_index(
#     drop=True
# )


print(game_time_df.shape)
game_time_df.head(2)

(21678, 18)


,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes,is_playoff,season_end_year,season_url,link,season_start_year,date,day_of_week
0,"Tue, Oct 30, 2007",8:00p,Portland Trail Blazers,97.0,San Antonio Spurs,106.0,Box Score,NaN,18797.0,AT&T Center,NaN,0,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007,2007-10-30,Tue
1,"Tue, Oct 30, 2007",10:30p,Utah Jazz,117.0,Golden State Warriors,96.0,Box Score,NaN,19832.0,Oracle Arena,NaN,0,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007,2007-10-30,Tue


In [20]:
game_time_df.columns

Index(['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1',
       'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes', 'is_playoff',
       'season_end_year', 'season_url', 'link', 'season_start_year', 'date',
       'day_of_week'],
      dtype='object')

In [21]:
keeps = [
    "day_of_week",
    "date",
    "Start (ET)",
    # "Visitor/Neutral",
    # "Home/Neutral",
    # "Arena",
    # "link",
    "season_start_year",
    "is_playoff",
]

away = [
    "Visitor/Neutral",
    "Home/Neutral",
]
home = [
    "Home/Neutral",
    "Visitor/Neutral",
]

home_teams = (
    game_time_df[home + keeps]
    .copy()
    .rename(columns={"Home/Neutral": "team", "Visitor/Neutral": "opponent"})
)
home_teams["is_home_game"] = 1
away_teams = (
    game_time_df[away + keeps]
    .copy()
    .rename(columns={"Home/Neutral": "opponent", "Visitor/Neutral": "team"})
)
away_teams["is_home_game"] = 0

print(game_time_df.shape)
print(home_teams.shape)
print(away_teams.shape)

(21678, 18)
(21678, 8)
(21678, 8)


In [22]:
combined = pd.concat([home_teams, away_teams])

combined = combined.sort_values(by=["season_start_year", "team", "date"]).reset_index(
    drop=True
)

# Convert 'Start (ET)' column to datetime
combined['game_time'] = pd.to_datetime(combined['date'] + ' ' + combined['Start (ET)'])

# # Calculate the time difference
# combined['Time_Diff'] = combined['game_time'].diff()

print(combined.shape)
combined.head(85)

/var/folders/dq/n1gz5xz55cj7qv8f04fcsb0m0000gp/T/ipykernel_56365/908911704.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined['game_time'] = pd.to_datetime(combined['date'] + ' ' + combined['Start (ET)'])


(43356, 9)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time
0,Atlanta Hawks,Dallas Mavericks,Fri,2007-11-02,7:30p,2007,0,1,2007-11-02 19:30:00
1,Atlanta Hawks,Detroit Pistons,Sun,2007-11-04,7:30p,2007,0,0,2007-11-04 19:30:00
2,Atlanta Hawks,New Jersey Nets,Tue,2007-11-06,7:30p,2007,0,0,2007-11-06 19:30:00
3,Atlanta Hawks,Phoenix Suns,Wed,2007-11-07,7:00p,2007,0,1,2007-11-07 19:00:00
4,Atlanta Hawks,Boston Celtics,Fri,2007-11-09,7:30p,2007,0,0,2007-11-09 19:30:00
...,...,...,...,...,...,...,...,...,...
80,Atlanta Hawks,Orlando Magic,Tue,2008-04-15,7:00p,2007,0,1,2008-04-15 19:00:00
81,Atlanta Hawks,Miami Heat,Wed,2008-04-16,7:30p,2007,0,0,2008-04-16 19:30:00
82,Atlanta Hawks,Boston Celtics,Sun,2008-04-20,8:30p,2007,1,0,2008-04-20 20:30:00
83,Atlanta Hawks,Boston Celtics,Wed,2008-04-23,8:00p,2007,1,0,2008-04-23 20:00:00


In [23]:
combined[
    (combined['date'] == "2007-11-02")
    & (combined['team'].isin(["Atlanta Hawks", "Dallas Mavericks"]))
]

,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time
0,Atlanta Hawks,Dallas Mavericks,Fri,2007-11-02,7:30p,2007,0,1,2007-11-02 19:30:00
457,Dallas Mavericks,Atlanta Hawks,Fri,2007-11-02,7:30p,2007,0,0,2007-11-02 19:30:00


In [24]:
# how many unique teams did they have each season?
team_count_piv = combined.pivot_table(
    index=["season_start_year"],
    values="team",
    aggfunc=pd.Series.nunique
)
team_count_piv

,team
season_start_year,
2007,30
2008,30
2009,30
2010,30
2011,30
2012,30
2013,30
2014,30
2015,30


In [25]:
# pause notebook
assert False

# save datafile
import time
combined.to_csv("../data/br_all_games.csv", index=False)
print(time.ctime())

AssertionError: 

In [26]:
# reg_season from 2014 onward
temp = combined[(combined["season_start_year"] >= 2014) & (combined["is_playoff"] == 0)].copy()
print(temp.shape)

temp.to_csv("../data/2014-2024_reg_season_raw.csv", index=False)

temp.head()


(23898, 9)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time
17922,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00
17923,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00
17924,Atlanta Hawks,San Antonio Spurs,Wed,2014-11-05,8:30p,2014,0,0,2014-11-05 20:30:00
17925,Atlanta Hawks,Charlotte Hornets,Fri,2014-11-07,7:00p,2014,0,0,2014-11-07 19:00:00
17926,Atlanta Hawks,New York Knicks,Sat,2014-11-08,7:30p,2014,0,1,2014-11-08 19:30:00


In [ ]:
# games per season per team
a = temp.pivot_table(
    index=['season_start_year', ],
    values=["team"],
    aggfunc="count"
)
a/30

,team
season_start_year,
2014,82.0
2015,82.0
2016,82.0
2017,82.0
2018,82.0
2019,70.6
2020,72.0
2021,82.0
2022,82.0
